In this notebook, we'll explore the basics of token representations in BERT and use it to find token nearest neighbors.  Before using, be sure to install the following libraries:

```sh
conda install -c conda-forge ipywidgets
conda install -c huggingface transformers
```


In [1]:
from transformers import BertModel, BertTokenizer
import numpy as np

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT uses WordPiece tokenization, which breaks down words that don't appear within its 30K-word vocabulary into small pieces.  The word "vaccinated", for instanced, is tokenized as ["va", "##cci", "##nated"]

In [3]:
inputs=tokenizer("New data shows 26 states have fully vaccinated more than half their residents.", return_tensors="pt")
tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

['[CLS]',
 'new',
 'data',
 'shows',
 '26',
 'states',
 'have',
 'fully',
 'va',
 '##cci',
 '##nated',
 'more',
 'than',
 'half',
 'their',
 'residents',
 '.',
 '[SEP]']

In [4]:
inputs=tokenizer("BERT is supercalifragilisticexpialidocious", return_tensors="pt")
tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

['[CLS]',
 'bert',
 'is',
 'super',
 '##cal',
 '##if',
 '##rag',
 '##ilis',
 '##tic',
 '##ex',
 '##pia',
 '##lid',
 '##oc',
 '##ious',
 '[SEP]']

In [17]:
inputs["input_ids"][0]

tensor([  101,  2023,  9389,  2003, 12090,   102])

As mentioned in class, notice how every sentence input to BERT is wrapped in two tags: a start [CLS] tag and an ending [SEP] tag.  BERT will generate representations of each WordPiece token, including these special [CLS] and [SEP] tags.

To generate representations for the input tokens, simply pass the input through the BERT model:

In [5]:
# take the tokens
inputs=tokenizer("This jam is delicious", return_tensors="pt")
# pass in the tokens to the model
outputs = model(**inputs)
tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

['[CLS]', 'this', 'jam', 'is', 'delicious', '[SEP]']

Representations for each of BERT layers (12 in this model) are accessible here, but let's explore just the outputs from the final layer.  This BERT model has 768-dimensional representations, so this 6-token input ([CLS, this, jam, is, delicious, [SEP]) has an output that is is a 1 x 6 tokens x 768 dimensional tensor.

In [6]:
last_hidden_states = outputs.last_hidden_state

In [7]:
print(outputs.last_hidden_state.shape)

torch.Size([1, 6, 768])


We'll cover using BERT for supervised problems in later classes, but what can we do with just these representations?  While we used word2vec-style static embeddings to find nearest neighbors for word *types*, we can do the same here for word *tokens*.

In [8]:
def cosine_similarity(a, b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [9]:
query="I ate some jam with toast"

In [10]:
comp_sents=["She got me out of a real jam", "This jam is made of strawberries", "I sat in a traffic jam for 2 hours", "The Grateful Dead used to jam for like two days straight.", "My grandma makes the best jam.", "I had to jam on the brakes to avoid hitting him."]


In [11]:
def get_bert_for_token(string, term):
    
    # tokenize
    inputs = tokenizer(string, return_tensors="pt")
    
    # convert input ids to words
    tokens=tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    
    # find the first location of the query term among those tokens (so we know which BERT rep to use)
    term_idx=tokens.index(term)
    
    outputs = model(**inputs)

    # return the BERT rep for that token index
    # The output is a pytorch tensor object, but let's convert it to a numpy object to work with numpy functions
    
    return outputs.last_hidden_state[0][term_idx].detach().numpy()

In [12]:
query_rep=get_bert_for_token(query, "jam")
print(query_rep.shape)

(768,)


In [13]:
vals=[]
for sent in comp_sents:
    comp_rep=get_bert_for_token(sent, "jam")
    cos_sim=cosine_similarity(query_rep, comp_rep)
    vals.append((cos_sim, query, sent))

for c, q, s in reversed(sorted(vals)):
    print("%.3f\t%s\t%s" % (c, q, s))

0.736	I ate some jam with toast	My grandma makes the best jam.
0.665	I ate some jam with toast	This jam is made of strawberries
0.480	I ate some jam with toast	I sat in a traffic jam for 2 hours
0.443	I ate some jam with toast	The Grateful Dead used to jam for like two days straight.
0.385	I ate some jam with toast	She got me out of a real jam
0.290	I ate some jam with toast	I had to jam on the brakes to avoid hitting him.


**Q**: Brainstorm the variety of things you can do with token-level word representations like this and we'll discuss them at the end of class.  As one example, adapt the code above to find *any* word that is most similar to *jam* in "I ate some jam with toast" in the following sentences.  Are you able to find token-level synonyms this way?

In [ ]:
comp_sents=["My grandma makes the best jelly.", "Jelly is made of strawberries"]